## Scraping job listings from Indeed

In [2]:
from time import sleep
import pandas as pd
import numpy as np
import requests
import bs4
from bs4 import BeautifulSoup
import urllib
import lxml
import re
import json

In [3]:
df = pd.DataFrame()
data = {}
counter = 0

keywords = ["Machine+Learning+Engineer", "algorithmic+trading", "financial+technology"
            "Data+Sales+Engineer", "Data+Consultant+Scientist", "Artificial+Intelligence", 
            "Natural+Language+Processing", "Computer+Vision", "Legal+techology",
            "Optimisation+engineer", "Pricing+engineer", "Quantitative+analyst+consultant"]
counter = 0

for keyword in keywords:
    for starter in range(0,2000,10): #from 0 to 2000 with interval of 10
        url = "https://au.indeed.com/jobs?q="+keyword+"&l=Melbourne+VIC&radius=75&start="+str(starter)
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'html.parser')
        pageresult = soup.find_all('div',{'class':'jobsearch-SerpJobCard'})
        for k in pageresult:
            try: title = k.find('a', attrs = {'data-tn-element': 'jobTitle'}).get_text().strip()
            except: title='NA'
            try: location = k.find('span', attrs = {'class': 'location'}).get_text().strip()
            except: location='Melbourne VIC'
            try: company = k.find('span', attrs = {'class':'company'}).get_text().strip()
            except: company = 'NA'
            try: salary = k.find('span', attrs = {'class': 'salary no-wrap'}).get_text().strip()
            except: salary = 'NA'
            try: 
                link= "https://au.indeed.com" + str(k.find('a', attrs = {'data-tn-element': 'jobTitle'}).get('href'))
                html2 = requests.get(link).text
                soup2 = BeautifulSoup(html2, 'html.parser')
                details = soup2.find('div',{'id': 'jobDescriptionText'}).get_text().strip()
            except: details='NA'
            data.update({counter: {"keyword":keyword, "location":location, "company": company, "title": title, "salary": salary, "description": details}})
            counter = counter + 1

In [4]:
with open('indeedMelbourneMay20.txt', 'w') as outfile:
    json.dump(data, outfile)

In [13]:
df = pd.DataFrame.from_dict(data).T

In [14]:
df.head()

,company,description,keyword,location,salary,title
0,Arq Group,Arq Group are Australia’s leading experts in M...,Machine+Learning+Engineer,Melbourne VIC,NA,Cloud Engineer
1,Avanade,Do you enjoy ensuring that information is avai...,Machine+Learning+Engineer,Melbourne VIC,NA,Azure Data Engineer
2,Servian,The opportunity\nInterested in joining one of ...,Machine+Learning+Engineer,Melbourne VIC,NA,Data Engineer
3,Zendesk,"As part of our engineering team, you will work...",Machine+Learning+Engineer,Melbourne VIC,NA,Data Engineer - Machine Learning Products
4,Zendesk,"As part of our engineering team, you will work...",Machine+Learning+Engineer,Melbourne VIC,NA,Applied Machine Learning Intern


In [8]:
len(data)

22047

In [15]:
# remove duplicates
salary = df.drop_duplicates(keep="first").dropna()
salary.location = 'melbourne'

In [16]:
# Converts other salaries into yearly salaries.
def salary_conversion(salarynumber):
    salarynumber = str(salarynumber).replace("$","").replace(",","").replace("-"," ")
    if "year" in salarynumber:
        newsal = np.mean([float(s) for s in salarynumber.split() if s.isdecimal()])
    elif "month" in salarynumber: 
        newsal = np.mean([float(s) for s in salarynumber.split() if s.isdecimal()])*12
    elif "week" in salarynumber: 
        newsal = np.mean([float(s) for s in salarynumber.split() if s.isdecimal()])*47            # assumption of 5 weeks annual leave and public holidays 52-5=47 weeks per year
    elif "day" in salarynumber:      # 47weeks*5days=235days per year
        newsal = np.mean([float(s) for s in salarynumber.split() if s.isdecimal()])*235
    elif "hour" in salarynumber:     # 
        newsal = np.mean([float(s) for s in salarynumber.split() if s.isdecimal()])*7.5*235
    else:
        newsal = "NA"
    return newsal

salary.loc[:,"salary"] = salary.loc[:,"salary"].map(salary_conversion)

salary = salary[salary.salary != 'NA']

C:\tools\Anaconda3\envs\tf_kernel\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\tools\Anaconda3\envs\tf_kernel\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [19]:
salary.to_csv('cleanIndeedMelbourneMay20.csv', index=False, encoding='utf-8')

In [18]:
salary

,company,description,keyword,location,salary,title
7,CSIRO,The Opportunity\nImmerse Yourself in an Inclus...,Machine+Learning+Engineer,melbourne,102000,Research Scientist - Data Integration
9,Crypto Recruit,Company:\nThe company is a cutting edge crypto...,Machine+Learning+Engineer,melbourne,125000,Data Scientist / Machine Learning (blockchain)
11,Akceli Pty Ltd,Job DescriptionBackend Engineer (Integration) ...,Machine+Learning+Engineer,melbourne,120000,"Backend Engineer (integration) - GraphQL, Expr..."
30,Redwolf + Rosch,Want to work on the latest tech/ML and join a ...,Machine+Learning+Engineer,melbourne,80000,JavaScript Engineer
34,IBM,Job Description\nSoftware Engineer – Cognitive...,Machine+Learning+Engineer,melbourne,87500,Software Engineer – Cognitive Analytics IBM Me...
36,IBM,Job Description\nIBM Research Scientist - Heal...,Machine+Learning+Engineer,melbourne,89250,IBM Research Scientist - Health AI FTH 24 Mont...
44,Snap Talent International,Our client is a fast growing Australian innova...,Machine+Learning+Engineer,melbourne,175000,Data Scientist
48,Contino,About Contino:\nWe are a global transformation...,Machine+Learning+Engineer,melbourne,175000,Technical Principal
58,Spark Recruitment,Excellent training on offer\nMust hold a degre...,Machine+Learning+Engineer,melbourne,75000,Cyber Security Presales Engineer - Distinction...
62,IBM,Job Description\nIBM Cloud Garage Software Dev...,Machine+Learning+Engineer,melbourne,87500,IBM Cloud Garage Software Developer – Melbourne


__________________


# Indeed Beijing

In [9]:
df_beijing = pd.DataFrame()
keywords = ["Data+Science", "Data+Scientist", "Machine+Learning+Engineer", 
           "Data+Sales+Engineer", "Data+Engineer", "Data+Consultant", "Artificial+Intelligence+Engineer", 
           "Machine+Learning+Consultant", "Natural+Language+Processing", "Computer+Vision", "NLP", "计算机视觉",
            "语音识别", "机器学习"]
cities = ["北京市", "深圳", "广州"]

for keyword in keywords:
    for city in cities:
        for starter in range(0,2000,10): #from 0 to 2000 with interval of 10
            url = "https://cn.indeed.com/jobs?q=" + keyword + "&l=" + city +"&start="+str(starter)
            html = requests.get(url).text
            soup = BeautifulSoup(html, 'html.parser')
            pageresult = soup.find_all('div',{'class':'jobsearch-SerpJobCard'})
            for k in pageresult:
                try: title = k.find('a', attrs = {'data-tn-element': 'jobTitle'}).get_text().strip()
                except: title='NA'
                try: location = k.find('span', attrs = {'class': 'location'}).get_text().strip()
                except: location='China'
                try: company = k.find('span', attrs = {'class':'company'}).get_text().strip()
                except: company = cities[city]
                try: salary = k.find('span', attrs = {'class': 'salary no-wrap'}).get_text().strip()
                except: salary = 'NA'
                try: description = k.find('div', attrs = {'class': 'summary'}).get_text().strip()
                except: description = 'NA'
                df_beijing = df_beijing.append({"location":location,"company":company, "title": title, "salary": salary, "description": description}, ignore_index=True) 

KeyboardInterrupt: 

In [ ]:
df_beijing.to_csv('indeedChinaMay17.csv', index=False, encoding='utf-8')

In [ ]:
df_beijing.head()

In [ ]:
url = "https://www.career-jet.cn/title-"+keyword+"-jobs.html"
https://www.career-jet.cn/wsearch/jobs?s=title+data+science&l=China&lid=84086&b=21
https://www.career-jet.cn/wsearch/jobs?s=title+data+science&l=China&lid=84086&b=41

df_china = pd.DataFrame()
keywords = ["Data+Science", "Data+Scientist", "Machine+Learning+Engineer", 
           "Data+Sales+Engineer", "Data+Engineer", "Data+Consultant", "Artificial+Intelligence+Engineer", 
           "Machine+Learning+Consultant", "Natural+Language+Processing", "Computer+Vision"]
cities = ["北京市", "深圳", "广州"]

for keyword in keywords:
    for starter in range(0,2000,10): #from 0 to 2000 with interval of 10
        url = "https://cn.indeed.com/jobs?q=北京市&l="+keyword+"&start="+str(starter)
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'html.parser')
        pageresult = soup.find_all('div',{'class':'jobsearch-SerpJobCard'})
        for k in pageresult:
            try: title = k.find('a', attrs = {'data-tn-element': 'jobTitle'}).get_text().strip()
            except: title='NA'
            try: location = k.find('span', attrs = {'class': 'location'}).get_text().strip()
            except: location='China'
            try: company = k.find('span', attrs = {'class':'company'}).get_text().strip()
            except: company = cities[city]
            try: salary = k.find('span', attrs = {'class': 'salary no-wrap'}).get_text().strip()
            except: salary = 'NA'
            try: description = k.find('div', attrs = {'class': 'summary'}).get_text().strip()
            except: description = 'NA'
            df_beijing = df_beijing.append({"location":location,"company":company, "title": title, "salary": salary, "description": description}, ignore_index=True) 

In [ ]:
for keyword in keywords:
    for starter in range(0,2000,10): #from 0 to 2000 with interval of 10
        url = "https://cn.indeed.com/jobs?q=北京市&l="+keyword+"&start="+str(starter)
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'html.parser')
        pageresult = soup.find_all('div',{'class':'jobsearch-SerpJobCard'})
        for k in pageresult:
            try: title = k.find('a', attrs = {'data-tn-element': 'jobTitle'}).get_text().strip()
            except: title='NA'
            try: location = k.find('span', attrs = {'class': 'location'}).get_text().strip()
            except: location='China'
            try: company = k.find('span', attrs = {'class':'company'}).get_text().strip()
            except: company = cities[city]
            try: salary = k.find('span', attrs = {'class': 'salary no-wrap'}).get_text().strip()
            except: salary = 'NA'
            try: description = k.find('div', attrs = {'class': 'summary'}).get_text().strip()
            except: description = 'NA'
            df_beijing = df_beijing.append({"location":location,"company":company, "title": title, "salary": salary, "description": description}, ignore_index=True) 

In [103]:
df_beijing.shape

(0, 0)

In [ ]:
df_beijing.to_csv('indeedBeijing')

# scrap ----> linked in

In [ ]:
熟悉NLP\计算机视觉\语音识别\虚拟助理\机器学习等AI相关领域


In [108]:
url = "https://www.career-jet.cn/title-"+keyword+"-jobs.html"
https://www.career-jet.cn/wsearch/jobs?s=title+data+science&l=China&lid=84086&b=21
https://www.career-jet.cn/wsearch/jobs?s=title+data+science&l=China&lid=84086&b=41
"https://cn.indeed.com/jobs?q=北京市&l="+"data+scientist"+"&start=0"
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')
pageresult = soup.find_all('div',{'class':'jobsearch-SerpJobCard'})
for k in pageresult:
    try: title = k.find('a', attrs = {'data-tn-element': 'jobTitle'}).get_text().strip()
    except: title='NA'
    try: location = k.find('span', attrs = {'class': 'location'}).get_text().strip()
    except: location='China'
    try: company = k.find('span', attrs = {'class':'company'}).get_text().strip()
    except: company = cities[city]
    try: salary = k.find('span', attrs = {'class': 'salary no-wrap'}).get_text().strip()
    except: salary = 'NA'
    try: description = k.find('div', attrs = {'class': 'summary'}).get_text().strip()
    except: description = 'NA'
    df_beijing = df_beijing.append({"location":location,"company":company, "title": title, "salary": salary, "description": description}, ignore_index=True) 

In [112]:
soup

<!DOCTYPE html>

<html dir="ltr" lang="zh">
<head>
<meta content="text/html;charset=utf-8" http-equiv="content-type"/>
<script src="/s/6098f96/zh_CN.js" type="text/javascript"></script>
<link href="/s/97464e7/jobsearch_all.css" rel="stylesheet" type="text/css"/>
<style type="text/css">
body,td{line-height:1.5;font-size:12px;}.jobtitle a, a.jobtitle {font-size:11pt;}.snip {width:50em}.jobtitle b,.company b,.location b,.summary b,.slc b{color:#c00;font-weight:normal;}a, a:link, .link, .btn, .btn:hover { text-decoration:none; }
a:hover, .link:hover { text-decoration:underline; }
.dya-container a { text-decoration: underline!important; }</style><link href="http://cn.indeed.com/rss?q=%E5%8C%97%E4%BA%AC%E5%B8%82&amp;l=data+scientist" rel="alternate" title="所有北京市招聘信息" type="application/rss+xml"/>
<link href="/m/jobs?q=%E5%8C%97%E4%BA%AC%E5%B8%82&amp;l=data+scientist" media="only screen and (max-width: 640px)" rel="alternate"/>
<link href="/m/jobs?q=%E5%8C%97%E4%BA%AC%E5%B8%82&amp;l=data+scien

In [85]:
df_linkedIn = pd.DataFrame()
keywords = ["Data%20Science", "Data%20Scientist", "Machine%20Learning%Engineer", 
           "Data%20Sales%20Engineer", "Data%20Engineer", "Data%20Consultant", "Artificial%20Intelligence%20Engineer", 
           "Machine%20Learning%Consultant", "Natural%20Language+Processing", "Computer%20Vision"]
   
for keyword in keywords:
    for starter in range(0,2000,25): #from 0 to 2000 with interval of 25
        url = "https://www.linkedin.com/jobs/search/?keywords="+str(keyword)+"&start="+str(starter)
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'html.parser')
        pageresult = soup.find_all('div',{'class': 'occludable-update artdeco-list__item'})
        for k in pageresult:
            try: title = Selector(text=k).xpath('//*[@id="ember117"]/text()').extract() 
                #title = k.find('a', attrs = {'id': 'ember3378'}).get_text().strip()
            except: title='NA'
#             try: location = k.find('span', attrs = {'class': 'location'}).get_text().strip()
#             except: 
            location='NA'
            try: company = k.find('a', attrs = {'class': "job-card-search__company-name-link"}).get_text().strip()      
            #try: company = k.find('a', attrs = {'class': "job-card-search__company-name-link ember-view"}).get_text().strip()      
            except: company = 'NA'
#             try: salary = k.find('span', attrs = {'class' : 'salary no-wrap'}).get_text().strip()
#             except: 
            salary = 'NA'
            try: description = k.find('p', attrs = {'class': 'job-card-search_source__link-wrapper'}).get_text().strip()
            except: description = 'NA'
            df_linkedIn = df_linkedIn.append({"location":location,"company":company, "title": title, "salary": salary}, ignore_index=True) 

df_linkedIn

""


Abandoned because security measures on linked-in